In [1]:
import sys
import polars as pl

package_paths = [
    '/kaggle/input/enefit-experiment-benchmark-all/kaggle_enefit_energy',
]

for pth in package_paths:
    sys.path.append(pth)

In [2]:
import enefit

from src.utils.import_utils import import_config, import_params
from src.preprocess import PreprocessPipeline
from src.model.lgbm import ModelPipeline

config_dict = {
    "PATH_ORIGINAL_DATA": r'/kaggle/input/predict-energy-behavior-of-prosumers',
    "PATH_PARQUET_DATA": None,
    "PATH_MAPPING_DATA": r'/kaggle/input/predict-energy-behavior-of-prosumers',
    "PATH_EXPERIMENT": r'/kaggle/input/enefit-experiment-benchmark-all/kaggle_enefit_energy/experiment',
    "N_FOLD": 5,
    "TARGET_COL": "target"
}

trainer = ModelPipeline(
    experiment_name='final_lgb',
    params_lgb={},
    config_dict=config_dict,
    metric_eval='l1', 
)

data_processor = PreprocessPipeline(
    config_dict=config_dict, 
    target_n_lags=14,
    embarko_skip=30
)

trainer.activate_inference()
data_processor.begin_inference()

In [3]:
env = enefit.make_env()
iter_test = env.iter_test()

In [4]:
for (
    test_data, 
    target_data_new, 
    client_data_new, 
    historical_weather_data_new,
    forecast_weather_data_new, 
    electricity_data_new, 
    gas_data_new, 
    sample_prediction
) in iter_test:

    data_processor.update_with_new_data(
        client_data_new = client_data_new,
        gas_data_new = gas_data_new,
        electricity_data_new = electricity_data_new,
        forecast_weather_data_new = forecast_weather_data_new,
        historical_weather_data_new = historical_weather_data_new,
        target_data_new = target_data_new,
        test_data = test_data
    )

    if any(test_data["currently_scored"]):
        data_processor()
        
        #ensure correct order of prediction
        sample_prediction["target"] = trainer.predict(data_processor.data)
                
    else:
        sample_prediction["target"] = 0
        
    env.predict(sample_prediction)    

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
